### STEP 5: NGROK  
- ngrok creates a public tunnel to expose local WebSocket server running on port 8111.  
- Converts the public HTTP URL into a WebSocket URL (`ws://` or `wss://`) so the UI can connect from anywhere.  


In [3]:
# Set up ngrok with your authtoken
from pyngrok import ngrok, conf
import time
port = 8111
# Set your authtoken
ngrok.set_auth_token("32JMoeiG3XmHLZ0N0LkRNWj3sDW_2C1jeFPh6gA45r9vE3ijZ")

# Kill any existing tunnels
ngrok.kill()


public_url = ngrok.connect(port)
print(f"🌐 Public URL: {public_url}")

# Convert to WebSocket URL
ws_url = str(public_url).replace("http://", "ws://").replace("https://", "wss://")
print(f"🔗 WebSocket URL: {ws_url}")

print("\n✅ Your TTS server is now publicly accessible!")
print("Anyone can connect to:", ws_url)

🌐 Public URL: NgrokTunnel: "https://948e667bc686.ngrok-free.app" -> "http://localhost:8111"
🔗 WebSocket URL: NgrokTunnel: "wss://948e667bc686.ngrok-free.app" -> "ws://localhost:8111"

✅ Your TTS server is now publicly accessible!
Anyone can connect to: NgrokTunnel: "wss://948e667bc686.ngrok-free.app" -> "ws://localhost:8111"


### Main Flow of the UI and Backend

1. **Connect**  
   - UI opens a WebSocket connection to the backend (`ws://...`).  
   - Sends a small `" "` message to initialize the session.  

2. **Send text**  
   - When you click *Generate Speech*, the UI sends JSON:  
     ```json
     { "text": "Hello world", "flush": true }
     ```  
   - `"flush": true` tells the backend to process the entire buffer now.
     

3. **Backend generates audio**  
   - Server splits text into chunks.  
   - For each chunk → runs TTS → returns **Base64 audio + alignment (timings)**.  

4. **UI receives response**  
   - Each response includes:  
     - `"audio"` (Base64-encoded speech chunk).  
     - `"alignment"` (characters + timestamps).  
   - UI logs the message, shows captions, and prepares audio.  

5. **Audio queueing**  
   - Base64 audio is decoded → turned into a WAV blob.  
   - Added to an **audioQueue** (so multiple chunks play in sequence).  

6. **Playback**  
   - If nothing is playing, the first audio chunk is loaded into `<audio>`.  
   - When one chunk finishes (`onended`), the next one in the queue plays automatically.  

7. **Captions in sync**  
   - Captions highlight characters using the timestamps.  
   - This happens while the audio is playing, not just when data arrives.  

➡️ **In short**:  
UI sends text → backend generates speech → UI queues audio + captions → plays chunks in order with real-time highlighting.  
